
## Tarea y actividad en clase.

1. Hacer merge de la rama que trabajamos a main.
2. Crear una nueva rama que se llame `feat: tarea 5`.
3. Crear un nuevo `jupyter-notebook` llamado `challenger-experiments.ipynb` en la rama creada anteriormente
4. Hacer dos `parent experiments` con `Gradient Boost` y `Random Forest` regressors en donde cada uno tenga `child experiments` con búsqueda de hyper-parámetros. Puede usar cualquier libreraría con la que se sienta cómodo: `hyperopt`, `optuna`, `scikit-learn` (Grid Search, Random Search, Halving Search etc)
5. Registrar el modelo con la mejor métrica `validation-rmse` de los obtenidos en dichos experimentos en el `model registry` en el mismo modelo ya previamente creado `nyc-taxi-model`.
6. Asígnele el alias `challenger`
7. Descargue en la carpeta `data` el conjunto de datos correspondiente a marzo del 2025
9. Use ese conjunto de datos para probarlo sobre los modelos con el alias `champion` y `challenger`
10. Obtenga la métrica de cada modelo
11. Decida si el nuevo modelo `challenger` debe ser promovido a `champion` o no. Use los criterios que usted como Data Scientis considere relevantes y justifique la respuesta.
12. Abrir un `PR` con los cambios hechos en la rama `feat: tarea 5` hacia la rama `main`.


Habrá dos entregas divididas de la siguiente manera:

1. **Trabajo en clase hoy Martes 21 de Octubre de 2025.** Para esta entrega, hacer un commit con el siguiente mensaje `feat: entrega trabajo en clase` con los avances realizados en clase.

2. **Tarea: Martes 28 de Octubre de 2025 a las 19:55.** Esta entrega debe contener todo lo descrito anteriormente

In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
import mlflow

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('data/green_tripdata_2025-02.parquet')

Feature Engineering + One Hot Encoding

In [4]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

Target

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los `dataset` como objetos de `mlflow` para poderlos trackear

In [7]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

## 🔧 Tunning de Hiper-parámetros para un modelo `xgboost` - Optuna

El **tunning de hiper-parámetros** consiste en encontrar la mejor combinación de parámetros que optimizan el rendimiento de un modelo.

En lugar de probar valores manualmente, usamos librerías como **Optuna**, que aplican estrategias inteligentes de búsqueda (*Bayesian optimization*, *Tree-structured Parzen Estimator*, etc.) para acelerar el proceso y encontrar resultados más robustos.

En este caso, usaremos **Optuna** para ajustar un modelo de **XGBoost**, definiendo un espacio de búsqueda para parámetros como:
| Parámetro | Descripción |
|------------|--------------|
| `max_depth` | Profundidad máxima de los árboles | 
| `learning_rate` | Tasa de aprendizaje (escala logarítmica) | 
| `reg_alpha` | Regularización L1 (α) | 
| `reg_lambda` | Regularización L2 (λ) | 
| `min_child_weight` | Peso mínimo de muestras por hoja | 
| `objective` | Función objetivo | 
| `seed` | Semilla aleatoria | 

Durante el proceso, Optuna crea un **“study”** donde cada *trial* representa una combinación de parámetros probada.  

Cada *trial* puede registrarse con **MLflow**, lo que permite visualizar métricas, comparar resultados y seleccionar el modelo más prometedor.

Ejemplo simplificado de estructura:

```python
import math, optuna, xgboost as xgb, mlflow

def objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",
        "seed": 42,
    }

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        booster = xgb.train(params, dtrain, evals=[(dvalid, "validation")])
        preds = booster.predict(dvalid)
        rmse = mean_squared_error(y_valid, preds, squared=False)
        mlflow.log_metric("rmse", rmse)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
```

🔹 **Ventajas de usar Optuna con MLflow:**
- Seguimiento automático de métricas y parámetros.  
- Comparación visual de resultados en la UI de MLflow.  
- Integración fluida con Databricks y notebooks colaborativos.  

---

📚 **Referencia interesante:**  
👉 [https://xgboosting.com/](https://xgboosting.com/) — Guía práctica con ejemplos avanzados de *tunning* y técnicas modernas para modelos de **XGBoost** y **Optuna**.


In [10]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

### Función Objetivo

In [12]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

### Flujo de búsqueda

In [15]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-28 18:36:04,530] A new study created in memory with name: no-name-1c29518c-e290-4068-a510-ae5188cbb4e7


MlflowException: Could not find experiment with ID 0

In [16]:
model_name = "workspace.default.nyc-taxi-model"

In [17]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

NameError: name 'EXPERIMENT_NAME' is not defined

In [18]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

NameError: name 'best_run' is not defined

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()

In [ ]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

## 🛠️ Anexo: cómo resolver errores de OpenMP (`libomp` / `libgomp`) con XGBoost

Cuando entrenas XGBoost, a veces aparecen errores tipo **“library not loaded: libomp.dylib”** (macOS) o **“cannot open shared object file: libgomp.so.1”** (Linux) o problemas con **vcomp / OpenMP** (Windows). Aquí tienes soluciones por sistema operativo.

---

### 🍏 macOS (Intel y Apple Silicon)
**Síntoma común:**  
`OSError: dlopen(... libxgboost.dylib ...): Library not loaded: @rpath/libomp.dylib`

**Solución rápida con Homebrew (recomendado):**
```bash
# Instalar / reinstalar OpenMP
brew update
brew install libomp || brew reinstall libomp

# (opcional) Si sigues con el error en notebooks lanzados desde terminal:
# export DYLD_LIBRARY_PATH antes de abrir Jupyter
echo 'export DYLD_LIBRARY_PATH="/opt/homebrew/opt/libomp/lib:$DYLD_LIBRARY_PATH"' >> ~/.zshrc
source ~/.zshrc
```

**Alternativas:**
- Si usas **Conda**: `conda install -c conda-forge xgboost libomp`  
  (las builds de conda-forge suelen traer las dependencias resueltas).
- En **Jupyter**, reinicia el kernel después de instalar `libomp`.
- Si el consumo de hilos es demasiado alto en laptops:  
  `export OMP_NUM_THREADS=4` (ajusta a tus cores).

---

### 🪟 Windows (PowerShell / CMD)
**Síntomas comunes:**
- Errores de OpenMP/vcomp o fallos al cargar `xgboost.dll`.
- En WSL (Ubuntu) ver la sección Linux.

**Solución:**
1. Asegúrate de usar **x64** y Python de 64 bits.
2. Instala/actualiza el **Microsoft Visual C++ Redistributable (2015–2022)**.  
   (Busca “Microsoft Visual C++ Redistributable x64” en el sitio oficial de Microsoft e instálalo).
3. Reinstala XGBoost:
   ```powershell
   pip install --upgrade --force-reinstall xgboost
   ```
4. Si tu CPU es de pocos núcleos o ves uso excesivo:
   ```powershell
   setx OMP_NUM_THREADS 4
   ```
   (Cierra y abre la terminal para que aplique).

**Notas:**
- Si usas **Conda**: `conda install -c conda-forge xgboost` suele resolver las DLLs.
- En entornos corporativos restringidos, ejecuta terminal como **Administrador** para el redistributable.

---

### 🐧 Linux (Ubuntu/Debian, Fedora, Alpine, Docker)
**Síntoma común:**  
`ImportError: libgomp.so.1: cannot open shared object file: No such file or directory`

**Solución (Ubuntu/Debian):**
```bash
sudo apt-get update
sudo apt-get install -y libgomp1
```

**Fedora/CentOS/RHEL:**
```bash
sudo dnf install -y libgomp   # o
sudo yum install -y libgomp
```

**Alpine (musl):**
```bash
sudo apk add libgomp
```

**Docker (por ejemplo, python:3.11-slim):**
```dockerfile
FROM python:3.11-slim
RUN apt-get update && apt-get install -y --no-install-recommends libgomp1 && rm -rf /var/lib/apt/lists/*
RUN pip install xgboost
```

**WSL (Ubuntu):**  
Sigue los pasos de Ubuntu/Debian y reinicia el kernel/notebook.

**Control de hilos (opcional):**
```bash
export OMP_NUM_THREADS=4
```